In [2]:
from tick_trade_api.api import TradeAPI
import pandas as pd
import datetime
import time
import math

# #清理所有多余文件
# import glob
# files=glob.glob('*.csv')
# for file in files:
#     if "行业" not in file:
#         remove_file(f"./{file}",trash=False)#删除所有文件
# files=glob.glob('*.ipynb')
# for file in files:
#     if "实盘" not in file:
#         remove_file(f"./{file}",trash=False)#删除所有文件

def symbol_convert(stock):#股票代码加后缀
    #北交所的股票8字开头，包括82、83、87、88，其中82开头的股票表示优先股；83和87开头的股票表示普通股票、88开头的股票表示公开发行的。
    if (stock.startswith("60"))or(#上交所主板
        stock.startswith("68"))or(#上交所科创板
        stock.startswith("11"))or(#上交所可转债
        (stock.startswith("51"))or(stock.startswith("56"))or(stock.startswith("58"))):#上交所ETF
        return str(str(stock)+".SH")
        # return str(str(stock)+".SS")
    elif (stock.startswith("00"))or(#深交所主板
        stock.startswith("30"))or(#深交所创业板
        stock.startswith("12"))or(#深交所可转债
        (stock.startswith("15"))):#深交所ETF
        return str(str(stock)+".SZ")
    else:
        print("不在后缀转换名录",str(stock))
        return str(str(stock))
    
def filter_kcb_stock(stocks):#过滤科创北交股票
    for stock in stocks[:]:
        if stock[0]=="4" or stock[0]=="8" or stock[:2]=="68":
            stocks.remove(stock)
    return stocks

def choose_stocks(choosename,now,start_date,last_date,today,yesterday):
    if choosename=="打板":#A股打板
        try:
            pd.read_csv(str(start_date)+"打板买入.csv")
            print("******",str(start_date)+"打板买入.csv"+"文件存在")
            pd.read_csv(str(start_date)+"打板卖出.csv")
            print("******",str(start_date)+"打板卖出.csv"+"文件存在")
        except Exception as e:
            print("******","******"+str(start_date)+"打板买入.csv"+"打板卖出.csv"+"文件不存在")
            while True:
                groupprice=pd.DataFrame({})
                thisnow=(datetime.datetime.utcnow()+datetime.timedelta(hours=8)).time()#获取标准时间，换算成东八区
                if thisnow>datetime.time(9,33):#【如果卡在这里了说明同花顺可能修改时区了】
                    print(thisnow,datetime.datetime.now())
                    groupprice=query_iwencai(
                        r"创业板或主板;非ST;前1日价跌量涨取反;今日涨停价≥30日最高收盘价或前10日最高收盘价≥前60日最高收盘价;昨日短期均线向上或昨日短期均线多头排列;9:30至9:32最高价/9:25价格≥1.03;9:30至9:32最低价/9:25价格≥0.990;竞价异动抢筹或9:25成交量/9:24委买≥1;(9:31分时量比*9点31分分时换手率)/(9:25分时量比*9点25分分时换手率)>5;2≤今日9:25分时量比≤30;-3%<=今日9:25分时涨跌幅<=3%;上一个交易日无涨停无炸板;今日9:25分时涨跌幅≥0%"
                        ,
                        domain='股票',timeout=6,df=True)
                    print(groupprice)
                    if len(groupprice)>0:
                        groupprice=groupprice.rename(columns={"股票代码":"代码","涨停价":"排名"})
                        groupprice["代码"]=groupprice["代码"].str.replace("\D","").astype(str)
                        groupprice.to_csv(str(start_date)+"打板买入.csv")
                        groupprice.to_csv(str(start_date)+"打板卖出.csv")
                        print("获取早盘打板数据",groupprice)
                    if groupprice.empty:
                        print("返回数据为空数据",datetime.datetime.now())
                        time.sleep(1)
                    if not groupprice.empty:
                        print("早盘打板选股结束")
                        break
                else:#【如果卡在这里了说明同花顺可能修改时区了】
                    print("未到开盘时间返回空",datetime.datetime.now())
                    time.sleep(1)
    elif choosename=="ETF（同花顺）":#A股聚宽ETF轮动策略
        try:
            pd.read_csv(str(start_date)+choosename+"买入.csv")
            print("******",str(start_date)+choosename+"买入.csv"+"文件存在")
            pd.read_csv(str(start_date)+choosename+"卖出.csv")
            print("******",str(start_date)+choosename+"卖出.csv"+"文件存在")
        except Exception as e:
            print("******","******"+str(start_date)+choosename+"买入.csv"+choosename+"卖出.csv"+"文件不同时存在")
            # df=get_all_securities('etf',today).reset_index(drop=False)
            # df=df.rename(columns={"symbol":"代码"})
            # df.to_csv("ETF.csv")
            etflist=[
                # "511880.SH",# 银华日利
                "518880.SH",#黄金ETF（大宗商品）备选方案"159934.SZ"
                "513100.SH",#纳指100（海外资产）备选方案"513100.SH"
                # "510300.SH",#沪深300
                # "510500.SH",#中证500
                # "510500.SH",#上证50
                "159949.SZ",#创业板50
                ]
            print(etflist)
            scoredf=pd.DataFrame({})
            for thisetf in etflist:
                df=get_price(
                            securities=thisetf,
                            start_date=None,
                            end_date=today,
                            fre_step="1d",
                            fields=["open"],
                            skip_paused=True,
                            fq="pre",
                            # fq=None,# 这里动态复权或者不复权都是可以的
                            bar_count=25,
                            is_panel=1,
                        )
                y=df["log"]=np.log(df.open)
                x=df["num"]=np.arange(df.log.size)
                slope,intercept=np.polyfit(x,y,1)
                annualized_returns=math.pow(math.exp(slope),250) - 1
                r_squared=1-(sum((y-(slope * x + intercept))**2) / ((len(y) - 1) * np.var(y,ddof=1)))
                score=annualized_returns * r_squared
                thisdf=pd.DataFrame({"代码":[thisetf],"score":[score]})
                scoredf=pd.concat([scoredf,thisdf])
            numbuystock=1
            print(scoredf)
            scoredf["排名"]=scoredf["score"].rank(method="max",ascending=False,na_option='bottom')
            scoredf["代码"]=scoredf["代码"].str.replace("\D","").astype(str)
            dftwo=scoredf.nlargest(math.ceil(1*numbuystock),"score")
            dfone=scoredf.nlargest(math.ceil(1*numbuystock),"score")
            dftwo.to_csv(str(start_date)+choosename+"卖出.csv")
            dfone.to_csv(str(start_date)+choosename+"买入.csv")
            dftwo["代码"]=dftwo["代码"].apply(lambda x:symbol_convert(x)).astype(str)#需要指定类型为字符串
            dfone["代码"]=dfone["代码"].apply(lambda x:symbol_convert(x)).astype(str)#需要指定类型为字符串
            buylisttwo=dftwo["代码"].values
            buylistone=dfone["代码"].values            
            print("******",buylistone,buylisttwo)
    elif choosename=="可转债（同花顺）":#A股可转债策略
        try:
            pd.read_csv(str(start_date)+choosename+"买入.csv")
            print("******",str(start_date)+choosename+"买入.csv"+"文件存在")
            pd.read_csv(str(start_date)+choosename+"卖出.csv")
            print("******",str(start_date)+choosename+"卖出.csv"+"文件存在")
        except Exception as e:
            print("******","******"+str(start_date)+choosename+"买入.csv"+choosename+"卖出.csv"+"文件不同时存在")
            # 获取所有可转债（当前可交易的）【这个函数回测的时候会报错】
            df=get_all_securities('cbond',today).reset_index(drop=False)
            print("当日可交易转债总数",len(df)) # 这里只有579只
            df=df.rename(columns={"symbol":"代码","display_name":"转债名称"})
            df=df[["代码","转债名称","end_date","start_date","market_hq"]]
            df=df[df["start_date"]<(datetime.datetime.strptime(yesterday,"%Y%m%d"))] # 去掉未上市的
            df=df[df["end_date"]>=(datetime.datetime.strptime(yesterday,"%Y%m%d")+datetime.timedelta(days=180))] # 去掉半年内到期的
            print("去掉即将到期的之后",len(df)) # 去掉即将退市的有559只
            # 获取全市场可转债余额变动表
            balancedf=run_query(query(cbond_remaining_change))
            balancedf=balancedf.rename(columns={
                "cbond_remaining_change_symbol":"代码",
                "cbond_remaining_change_change_date":"转债余额变更日",
                "cbond_remaining_change_remaining":"转债余额"}) # 转债余额的单位是万
            balancedf=balancedf[balancedf["转债余额变更日"]<(datetime.datetime.strptime(yesterday,"%Y%m%d")+datetime.timedelta(days=180)).date()] # 去掉180天内到期或者赎回的
            balancedf=balancedf.sort_values(by='转债余额变更日',ascending=False)
            balancedf=balancedf.groupby("代码").apply(lambda x: x[:1]) # 只保留最后一天
            balancedf=balancedf.reset_index(drop=True) # 重置索引
            balancedf=balancedf[["代码","转债余额变更日","转债余额"]]
            balancedf=balancedf[(balancedf["转债余额"]!=0)] # 单位是万元，这里其实是小于2.5亿
            # balancedf.to_csv("可转债余额变动表.csv")
            # # 获取全市场可转债转股价变更表【总共600只】
            ratepricedf=run_query(query(cbond_conversion_change))
            ratepricedf=ratepricedf.rename(columns={"cbond_conversion_change_symbol":"代码",
                                            "cbond_conversion_change_change_date":"转股价变更日",
                                            "cbond_conversion_change_report_date":"转股价变更公告日",
                                            "cbond_conversion_change_conversion_price":"更新后转股价格",
                                           }) # 转债余额的单位是万
            ratepricedf=ratepricedf.sort_values(by='转股价变更日',ascending=False)
            ratepricedf=ratepricedf.groupby("代码").apply(lambda x: x[:1]) # 只保留最后一天
            ratepricedf=ratepricedf.reset_index(drop=True) # 重置索引
            ratepricedf=ratepricedf[["代码","转股价变更日","转股价变更公告日","更新后转股价格"]]
            # ratepricedf.to_csv("可转债转股价变更表.csv")
            # # 可转债标的和正股标的对应关系
            df_cbonds=run_query(query(cbond_info))
            df_cbonds=df_cbonds[(df_cbonds["cbond_info_pub_enddate"]>=(datetime.datetime.strptime(yesterday,"%Y%m%d")+datetime.timedelta(days=180)).date())] # 转债结束日期在180天之后
            df_cbonds=df_cbonds.rename(columns={"cbond_info_symbol":"代码","cbond_info_stock_symbol":"正股代码"})
            df_cbonds=df_cbonds[["代码","正股代码","cbond_info_pub_enddate"]] # 这里只有448只
            df_cbonds=df_cbonds[(df_cbonds["代码"].str.startswith("12"))|df_cbonds["代码"].str.startswith("118")] # 只保留12开头的深债或者118开头的科创板债
            # 拼接之前的数据
            df_cbonds=df_cbonds.merge(df,on="代码",how="inner").reset_index(drop=True) # 拼接之前的可转债余额变动表
            df_cbonds=df_cbonds.merge(balancedf,on="代码",how="inner").reset_index(drop=True) # 拼接之前的可转债余额变动表
            df_cbonds=df_cbonds.merge(ratepricedf,on="代码",how="inner").reset_index(drop=True) # 拼接之前的可转债转股价变更表
            # 剔除正股ST的标的
            stocks=get_all_securities('stock',today).index.tolist()
            stocks=[stock for stock in stocks if ((stocks[0]!="4") and (stock[0]!="8"))] # 去掉北交所新三板股票【没有去688的科创板股票】
            stocks=[stock for stock in stocks if stock in df_cbonds["正股代码"].tolist()] # 提前去掉正股不符合基本面要求的标的
            df_stocks=get_price(
                securities=stocks,
                start_date=None,
                end_date=today,
                fre_step='1d',
                fields=["is_st"],# 获取全部数据列
                skip_paused=True,
                fq='pre',
                # fq=None,# 这里动态复权或者不复权都是可以的
                bar_count=1,
                is_panel=1,).to_frame()
            df_stocks=df_stocks[(df_stocks["is_st"]==0)] # 当天去除ST，但是不去除停牌，因为正股停牌不影响ST
            df_stocks=df_stocks.reset_index() # 重置索引
            df_stocks=df_stocks.rename(columns={"minor":"正股代码"})
            df_stocks=df_stocks[["正股代码"]]
            # 获取基本面数据【正股】
            olddf=get_fundamentals(query(
                asharevalue.symbol,
                asharevalue.total_shares,
                asharevalue.float_shares,#流通股本
                income.profit_before_tax,#利润总额【当计提历史亏损的时候利润总额大于当年净利润】
                income.net_profit,#净利润
                income.np_atsopc,#归母净利润
                income.overall_income,#营业总收入
                balance.undistributed_profits,#未分配利润
                balance.total_liabilities,#负债合计
                balance.total_liab_and_holders_equity,#负债和股东权益总计
                asharevalue.pb_mrq,#市净率MRQ
                ),date=yesterday)
            olddf=olddf.rename(columns={
                    "asharevalue_symbol":"正股代码",
                    "asharevalue_total_shares": "总股本",
                    # "asharevalue_float_shares": "流通股本",
                    "income_profit_before_tax":"利润总额",
                    "income_net_profit":"净利润",
                    "income_np_atsopc":"归母净利润",
                    "income_overall_income":"营业总收入",
                    "balance_undistributed_profits":"未分配利润",
                    # "income_overall_income":"营业总收入",
                    "balance_total_liabilities":"负债合计",
                    "balance_total_liab_and_holders_equity":"负债和股东权益总计",
                    "asharevalue_pb_mrq":"市净率",
                })
            olddf["资产负债率"]=olddf["负债合计"]/olddf["负债和股东权益总计"]
            olddf=olddf[olddf["资产负债率"]<1]#存在资产负债率大于1（资不抵债）的标的，跟历史数据一致
            olddf=olddf[olddf["利润总额"]>0]
            olddf=olddf[olddf["净利润"]>0]
            olddf=olddf[olddf["归母净利润"]>0]
            olddf=olddf[olddf["未分配利润"]>0]
            # olddf=olddf[olddf["营业总收入"]>100000000]
            olddf=olddf[["正股代码","归母净利润"]]
            df_stocks=df_stocks.merge(olddf,on="正股代码",how="inner")
            # 拼接正股数据
            df_cbonds=df_cbonds.merge(df_stocks,on="正股代码",how="inner")
            print("去掉正股ST的",len(df_cbonds))
            # 设置获取k线的时间
            timestame=(now-datetime.timedelta(minutes=2)).strftime("%Y%m%d %H:%M")
            # 获取债券一分钟k数据
            cbonds=df_cbonds["代码"].tolist()
            df_cbonds_price=get_price(
                securities=cbonds,
                start_date=None,

                # # 日级别数据
                # end_date=today,
                # fre_step='1d',
                # fields=["open"],

                # # 分钟级别数据
                end_date=timestame,
                fre_step='1m',
                fields=["open"],
                skip_paused=True,
                fq='pre',
                # fq=None,# 这里动态复权或者不复权都是可以的
                bar_count=1,
                is_panel=1,).to_frame()
            df_cbonds_price=df_cbonds_price.reset_index()
            df_cbonds_price=df_cbonds_price[df_cbonds_price["open"]<150]
            df_cbonds_price=df_cbonds_price.rename(columns={"minor":"代码","open":"可转债价格"})
            df_cbonds_price=df_cbonds_price[["代码","可转债价格"]]
            df_cbonds=pd.merge(df_cbonds,df_cbonds_price,on="代码")
            # 获取正股一分钟k数据
            stocks=df_cbonds["正股代码"].tolist()
            df_stocks_price=get_price(
                securities=stocks,
                start_date=None,

                # # 日级别数据
                # end_date=today,
                # fre_step='1d',
                # fields=["open"],

                # # 分钟级别数据
                end_date=timestame,
                fre_step='1m',
                fields=["open"],
                skip_paused=True,
                fq='pre',
                # fq=None,# 这里动态复权或者不复权都是可以的
                bar_count=1,
                is_panel=1,).to_frame()
            df_stocks_price=df_stocks_price.reset_index()
            df_stocks_price=df_stocks_price.rename(columns={"minor":"正股代码","open":"正股价格"})
            df_stocks_price=df_stocks_price[["正股代码","正股价格"]]
            df_cbonds=pd.merge(df_cbonds,df_stocks_price,on="正股代码")
            df_cbonds["转股溢价率"]=df_cbonds["可转债价格"]/((100/df_cbonds["更新后转股价格"])*df_cbonds["正股价格"])
            df_cbonds["总市值"]=df_cbonds["可转债价格"]*df_cbonds["转债余额"]/10000
            df_cbonds["三低指数"]=df_cbonds["总市值"]*df_cbonds["转股溢价率"]
            df_cbonds["排名"]=df_cbonds["三低指数"].rank(method="max",ascending=True,na_option='bottom')
            df_cbonds["代码"]=df_cbonds["代码"].str.replace("\D","").astype(str)

            numbuystock=5 # 设置持仓数量
            dftwo=df_cbonds.nsmallest(math.ceil(1.5*numbuystock),"三低指数")
            dfone=df_cbonds.nsmallest(math.ceil(numbuystock),"三低指数")

            dftwo.to_csv(str(start_date)+choosename+"卖出.csv")
            dfone.to_csv(str(start_date)+choosename+"买入.csv")
            dftwo["代码"]=dftwo["代码"].apply(lambda x:symbol_convert(x)).astype(str)#需要指定类型为字符串
            dfone["代码"]=dfone["代码"].apply(lambda x:symbol_convert(x)).astype(str)#需要指定类型为字符串
            buylisttwo=dftwo["代码"].values
            buylistone=dfone["代码"].values
            print("******",buylistone,buylisttwo)
    elif choosename=="大市值（同花顺）":#A股大市值策略
        try:
            pd.read_csv(str(start_date)+choosename+"买入.csv")
            print("******",str(start_date)+choosename+"买入.csv"+"文件存在")
            pd.read_csv(str(start_date)+choosename+"卖出.csv")
            print("******",str(start_date)+choosename+"卖出.csv"+"文件存在")
        except Exception as e:
            print("******","******"+str(start_date)+choosename+"买入.csv"+choosename+"卖出.csv"+"文件不同时存在")

            stocks=get_all_securities('stock',today)
            print("修改前",len(stocks))
            stocks=stocks[~stocks["display_name"].str.contains("退|ST")]
            # stocks=stocks[stocks["start_date"]<last_date] # 去掉次新股
            stocks=stocks.index.values # 获取当天所有标的
            print("修改后",len(stocks))
            stocks=[stock for stock in stocks]
            stocks=filter_kcb_stock(stocks) # 去除科创北交
            print(len(stocks))
            df=get_price(
                securities=stocks,
                start_date=None,
                end_date=today,
                fre_step='1d',
                fields=["open","is_paused","is_st","prev_close","high_limit","low_limit"],# 获取全部数据列，其中昨日收盘价是用来计算总股本的
                fq='pre',
                # fq=None,# 这里动态复权或者不复权都是可以的
                bar_count=1,
                is_panel=1).to_frame()
            df=df.reset_index()
            df=df[(df["is_paused"]==0)&(df["is_st"]==0)]
            # df=df[df["open"]>4]
            df=df.rename(columns={"minor": "代码","high_limit":"涨停价","low_limit":"跌停价"})
            df=df[["代码","open","涨停价","跌停价"]]
            # 获取基本面数据
            olddf=get_fundamentals(query(
                asharevalue.symbol,
                asharevalue.total_shares,
                asharevalue.float_shares,#流通股本
                income.profit_before_tax,#利润总额【当计提历史亏损的时候利润总额大于当年净利润】
                income.net_profit,#净利润
                income.np_atsopc,#归母净利润
                income.overall_income,#营业总收入
                balance.undistributed_profits,#未分配利润
                balance.total_liabilities,#负债合计
                balance.total_liab_and_holders_equity,#负债和股东权益总计
                asharevalue.pb_mrq,#市净率MRQ
                ),date=yesterday)
            olddf=olddf.rename(columns={
                    "asharevalue_symbol":"代码",
                    "asharevalue_total_shares": "总股本",
                    # "asharevalue_float_shares": "流通股本",
                    "income_profit_before_tax":"利润总额",
                    "income_net_profit":"净利润",
                    "income_np_atsopc":"归母净利润",
                    "income_overall_income":"营业总收入",
                    "balance_undistributed_profits":"未分配利润",
                    # "income_overall_income":"营业总收入",
                    "balance_total_liabilities":"负债合计",
                    "balance_total_liab_and_holders_equity":"负债和股东权益总计",
                    "asharevalue_pb_mrq":"市净率",
                })
            olddf["资产负债率"]=olddf["负债合计"]/olddf["负债和股东权益总计"]
            olddf=olddf[olddf["资产负债率"]<1]#存在资产负债率大于1（资不抵债）的标的，跟历史数据一致
            olddf=olddf[olddf["利润总额"]>0]
            olddf=olddf[olddf["净利润"]>0]
            olddf=olddf[olddf["归母净利润"]>0]
            olddf=olddf[olddf["未分配利润"]>0]
            # olddf=olddf[olddf["营业总收入"]>100000000]
            olddf=olddf.merge(df,on="代码",how="inner")
            olddf["总市值"]=olddf["总股本"]*olddf["open"]
            olddf=olddf.nlargest(20,"总市值")
            olddf["代码"]=olddf["代码"].str.replace("\D","").astype(str)
            print(olddf)
            numbuystock=5 # 设置持仓数量
            olddf["市净率"]=1/olddf["市净率"]
            olddf["排名"]=olddf["市净率"].rank(method="max",ascending=False,na_option='bottom')
            olddf["代码"]=olddf["代码"].str.replace("\D","").astype(str)
            dftwo=olddf.nlargest(math.ceil(2*numbuystock),"市净率")
            dfone=olddf.nlargest(math.ceil(numbuystock),"市净率")
            dftwo.to_csv(str(start_date)+choosename+"卖出.csv")
            dfone.to_csv(str(start_date)+choosename+"买入.csv")
            dftwo["代码"]=dftwo["代码"].apply(lambda x:symbol_convert(x)).astype(str)#需要指定类型为字符串
            dfone["代码"]=dfone["代码"].apply(lambda x:symbol_convert(x)).astype(str)#需要指定类型为字符串
            buylisttwo=dftwo["代码"].values
            buylistone=dfone["代码"].values
            print("******",buylistone,buylisttwo)
    elif choosename=="中小板（同花顺）":#A股中小板策略
        try:
            pd.read_csv(str(start_date)+choosename+"买入.csv")
            print("******",str(start_date)+choosename+"买入.csv"+"文件存在")
            pd.read_csv(str(start_date)+choosename+"卖出.csv")
            print("******",str(start_date)+choosename+"卖出.csv"+"文件存在")
        except Exception as e:
            print("******","******"+str(start_date)+choosename+"买入.csv"+choosename+"卖出.csv"+"文件不同时存在")
            stocks=get_all_securities('stock',today)
            print("修改前",len(stocks))
            stocks=stocks[~stocks["display_name"].str.contains("退|ST")]
            # stocks=stocks[stocks["start_date"]<last_date] # 去掉次新股
            stocks=stocks.index.values # 获取当天所有标的
            print("修改后",len(stocks))
            stocks=[stock for stock in stocks]
            stocks=filter_kcb_stock(stocks) # 去除科创北交
            print(len(stocks))
            thisstocks=get_index_stocks("399101.SZ",date=today)#中小综指
            stocks=[stock for stock in thisstocks if stock in stocks]
            df=get_price(
                securities=stocks,
                start_date=None,
                end_date=today,
                fre_step="1d",
                fields=["open","is_paused","is_st","prev_close"],#获取全部数据列，其中昨日收盘价是用来计算总股本的
                fq="pre",
                #fq=None,#这里动态复权或者不复权都是可以的
                bar_count=1,
                is_panel=1,
            ).to_frame()
            df=df.reset_index()
            df=df[(df["is_paused"]==0)&(df["is_st"]==0)]
            # df=df[df["open"]>4]
            df=df.rename(columns={"minor":"代码"})
            df=df[["代码","open"]]
            # 获取基本面数据
            olddf=get_fundamentals(query(
                asharevalue.symbol,
                asharevalue.total_shares,
                asharevalue.float_shares,#流通股本
                income.profit_before_tax,#利润总额【当计提历史亏损的时候利润总额大于当年净利润】
                income.net_profit,#净利润
                income.np_atsopc,#归母净利润
                income.overall_income,#营业总收入
                balance.undistributed_profits,#未分配利润
                balance.total_liabilities,#负债合计
                balance.total_liab_and_holders_equity,#负债和股东权益总计
                asharevalue.pb_mrq,#市净率MRQ
                ),date=yesterday)
            olddf=olddf.rename(columns={
                    "asharevalue_symbol":"代码",
                    "asharevalue_total_shares": "总股本",
                    # "asharevalue_float_shares": "流通股本",
                    "income_profit_before_tax":"利润总额",
                    "income_net_profit":"净利润",
                    "income_np_atsopc":"归母净利润",
                    "income_overall_income":"营业总收入",
                    "balance_undistributed_profits":"未分配利润",
                    # "income_overall_income":"营业总收入",
                    "balance_total_liabilities":"负债合计",
                    "balance_total_liab_and_holders_equity":"负债和股东权益总计",
                    "asharevalue_pb_mrq":"市净率",
                })
            olddf["资产负债率"]=olddf["负债合计"]/olddf["负债和股东权益总计"]
            olddf=olddf[olddf["资产负债率"]<1]#存在资产负债率大于1（资不抵债）的标的，跟历史数据一致
            olddf=olddf[olddf["利润总额"]>0]
            olddf=olddf[olddf["净利润"]>0]
            olddf=olddf[olddf["归母净利润"]>0]
            olddf=olddf[olddf["未分配利润"]>0]
            # olddf=olddf[olddf["营业总收入"]>100000000]
            olddf=olddf.merge(df,on="代码",how="inner")
            olddf["总市值"]=olddf["总股本"]*olddf["open"]
            #获取最近一分钟市值最小的标的
            olddf=olddf.nsmallest(math.ceil(100),"总市值")
            olddf=olddf[["代码","总股本","归母净利润"]]
            timestame=(now-datetime.timedelta(minutes=2)).strftime("%Y%m%d %H:%M")#设置获取k线的时间
            stocks=olddf["代码"].tolist()
            df=get_price(
                securities=stocks,
                start_date=None,

                # # 日级别数据
                # end_date=today,
                # fre_step='1d',
                # fields=["open"],

                # # 分钟级别数据
                end_date=timestame,
                fre_step='1m',
                fields=["open"],

                skip_paused=True,
                fq='pre',
                #fq=None,#这里动态复权或者不复权都是可以的
                bar_count=1,
                is_panel=1,).to_frame()
            df=df.reset_index()
            df=df.rename(columns={"minor":"代码"})
            olddf=olddf.merge(df,on="代码",how="inner")
            olddf["总市值"]=olddf["总股本"]*olddf["open"]
            olddf["排名"]=olddf["总市值"].rank(method="max",ascending=True,na_option='bottom')
            olddf["代码"]=olddf["代码"].str.replace("\D","").astype(str)
            numbuystock=6#设置持仓数量
            # numbuystock=10#设置持仓数量
            dftwo=olddf.nsmallest(math.ceil(1.5*numbuystock),"总市值")
            dfone=olddf.nsmallest(math.ceil(numbuystock),"总市值")
            dftwo.to_csv(str(start_date)+choosename+"卖出.csv")
            dfone.to_csv(str(start_date)+choosename+"买入.csv")
            dftwo["代码"]=dftwo["代码"].apply(lambda x:symbol_convert(x)).astype(str)#需要指定类型为字符串
            dfone["代码"]=dfone["代码"].apply(lambda x:symbol_convert(x)).astype(str)#需要指定类型为字符串
            buylisttwo=dftwo["代码"].values
            buylistone=dfone["代码"].values
            print("******",buylistone,buylisttwo)
    elif choosename=="微盘股（同花顺）":#A股微盘股策略
        try:
            pd.read_csv(str(start_date)+choosename+"买入.csv")
            print("******",str(start_date)+choosename+"买入.csv"+"文件存在")
            pd.read_csv(str(start_date)+choosename+"卖出.csv")
            print("******",str(start_date)+choosename+"卖出.csv"+"文件存在")
        except Exception as e:
            print("******","******"+str(start_date)+choosename+"买入.csv"+choosename+"卖出.csv"+"文件不同时存在")
            stocks=get_all_securities('stock',today)
            print("修改前",len(stocks))
            stocks=stocks[~stocks["display_name"].str.contains("退|ST")]
            # stocks=stocks[stocks["start_date"]<last_date] # 去掉次新股
            stocks=stocks.index.values # 获取当天所有标的
            print("修改后",len(stocks))
            stocks=[stock for stock in stocks]
            stocks=filter_kcb_stock(stocks) # 去除科创北交
            print(len(stocks))
            df=get_price(
                securities=stocks,
                start_date=None,
                end_date=today,
                fre_step='1d',
                fields=["open","is_paused","is_st","prev_close","high_limit","low_limit"],# 获取全部数据列，其中昨日收盘价是用来计算总股本的
                fq='pre',
                # fq=None,# 这里动态复权或者不复权都是可以的
                bar_count=1,
                is_panel=1).to_frame()
            df=df.reset_index()
            df=df[(df["is_paused"]==0)&(df["is_st"]==0)]
            # df=df[df["open"]>4]
            df=df.rename(columns={"minor": "代码","high_limit":"涨停价","low_limit":"跌停价"})
            df=df[["代码","open","涨停价","跌停价"]]
            # 获取基本面数据
            olddf=get_fundamentals(query(
                asharevalue.symbol,
                asharevalue.total_shares,
                asharevalue.float_shares,#流通股本
                income.profit_before_tax,#利润总额【当计提历史亏损的时候利润总额大于当年净利润】
                income.net_profit,#净利润
                income.np_atsopc,#归母净利润
                income.overall_income,#营业总收入
                balance.undistributed_profits,#未分配利润
                balance.total_liabilities,#负债合计
                balance.total_liab_and_holders_equity,#负债和股东权益总计
                asharevalue.pb_mrq,#市净率MRQ
                ),date=yesterday)
            olddf=olddf.rename(columns={
                    "asharevalue_symbol":"代码",
                    "asharevalue_total_shares": "总股本",
                    # "asharevalue_float_shares": "流通股本",
                    "income_profit_before_tax":"利润总额",
                    "income_net_profit":"净利润",
                    "income_np_atsopc":"归母净利润",
                    "income_overall_income":"营业总收入",
                    "balance_undistributed_profits":"未分配利润",
                    # "income_overall_income":"营业总收入",
                    "balance_total_liabilities":"负债合计",
                    "balance_total_liab_and_holders_equity":"负债和股东权益总计",
                    "asharevalue_pb_mrq":"市净率",
                })
            olddf["资产负债率"]=olddf["负债合计"]/olddf["负债和股东权益总计"]
            olddf=olddf[olddf["资产负债率"]<1]#存在资产负债率大于1（资不抵债）的标的，跟历史数据一致
            olddf=olddf[olddf["利润总额"]>0]
            olddf=olddf[olddf["净利润"]>0]
            olddf=olddf[olddf["归母净利润"]>0]
            olddf=olddf[olddf["未分配利润"]>0]
            # olddf=olddf[olddf["营业总收入"]>100000000]
            olddf=olddf.merge(df,on="代码",how="inner")
            olddf["总市值"]=olddf["总股本"]*olddf["open"]
            #获取最近一分钟市值最小的标的
            olddf=olddf.nsmallest(math.ceil(100),"总市值")
            olddf=olddf[["代码","总股本","归母净利润"]]
            timestame=(now-datetime.timedelta(minutes=2)).strftime("%Y%m%d %H:%M")#设置获取k线的时间
            stocks=olddf["代码"].tolist()
            df=get_price(
                securities=stocks,
                start_date=None,

                # # 日级别数据
                # end_date=today,
                # fre_step='1d',
                # fields=["open"],

                # # 分钟级别数据
                end_date=timestame,
                fre_step='1m',
                fields=["open"],

                skip_paused=True,
                fq='pre',
                #fq=None,#这里动态复权或者不复权都是可以的
                bar_count=1,
                is_panel=1,).to_frame()
            df=df.reset_index()
            df=df.rename(columns={"minor":"代码"})
            olddf=olddf.merge(df,on="代码",how="inner")
            olddf["总市值"]=olddf["总股本"]*olddf["open"]
            olddf["排名"]=olddf["总市值"].rank(method="max",ascending=True,na_option='bottom')
            olddf["代码"]=olddf["代码"].str.replace("\D","").astype(str)
            numbuystock=30 # 设置持仓数量
            dftwo=olddf.nsmallest(math.ceil(1.5*numbuystock),"总市值")
            dfone=olddf.nsmallest(math.ceil(numbuystock),"总市值")
            dftwo.to_csv(str(start_date)+choosename+"卖出.csv")
            dfone.to_csv(str(start_date)+choosename+"买入.csv")
            dftwo["代码"]=dftwo["代码"].apply(lambda x:symbol_convert(x)).astype(str)#需要指定类型为字符串
            dfone["代码"]=dfone["代码"].apply(lambda x:symbol_convert(x)).astype(str)#需要指定类型为字符串
            buylisttwo=dftwo["代码"].values
            buylistone=dfone["代码"].values
            print("******",buylistone,buylisttwo)

#获取交易日期【实盘的时候每天交易两次开盘、收盘各进行一次能够进一步提高收益】
now=datetime.datetime.now()
start_date=now.strftime("%Y-%m-%d")
last_date=(now-datetime.timedelta(days=250)).strftime("%Y-%m-%d")
print(start_date,last_date)

today=get_trade_days(end_date=start_date,count=2).values[1]
today=np.datetime_as_string(today,unit="D").replace("-","")
yesterday=get_trade_days(end_date=start_date,count=2).values[0]
preyesterday=np.datetime_as_string(yesterday,unit="D")
yesterday=preyesterday.replace("-","")
print("******","等待策略初始化",today,yesterday)

# account_id='15003582'#资金账号（实盘【没有资金】）
# account_id='15005978'#资金账号（实盘【有资金】）
# account_id='13020385102'#资金账号（手机模拟（戴庚霖）【执行ETF策略】
# account_id='15803281949'#资金账号（手机模拟（王恒柱）【执行可转债策略】
account_id='19511189162'#资金账号（手机模拟（王腾鹤）【执行中小板策略】
# account_id='90536336'#资金账号（千万模拟）【执行大市值策略】
# account_id='90536337'#资金账号（千万模拟）【执行微盘股策略】

#设置交易参数并且获取买卖计划
bidrate=0.005#设置盘口价差为0.004
timecancellwait=60#设置撤单函数筛选订单的确认时间
timetickwait=20#设置每次下单时确认是否是最新tick的确认时间
timeseconds=40#设置获取tick的函数的时间长度【避免没有数据】

targetmoney=25000#设置下单时对手盘需要达到的厚度（即单笔目标下单金额，因为手数需要向下取整，所以实际金额比这个值低）
traderate=2#设置单次挂单金额是targetmoney的traderate倍
# cancellorder=False#取消一分钟不成交或者已成交金额达到目标值自动撤单并回补撤单金额的任务
cancellorder=True#设置一分钟不成交或者已成交金额达到目标值自动撤单并回补撤单金额的任务

if (account_id=='15005978'):
    choosename="中小板（同花顺）"
    
    # thisnow=datetime.datetime.now().strftime('%m-%d')
    # if (('01-01'<=thisnow)and(thisnow<='01-30')):#一月空仓
    # # if (('04-01'<=thisnow)and(thisnow<='04-30'))or(('01-01'<=thisnow)and(thisnow<='01-30')):#一月四月空仓
    #     # choosename=""#空仓
    #     choosename="大市值（同花顺）"
    # else:
    #     choosename="中小板（同花顺）"
    
    tradeway="taker"#设置主动吃单
    # tradeway="maker"#设置被动吃单
elif (account_id=='13020385102'):
    choosename="ETF（同花顺）"
    tradeway="taker"#设置主动吃单
    # tradeway="集合竞价"【仅仅个股可以使用】
elif (account_id=='15803281949'):
    choosename="可转债（同花顺）"
    tradeway="taker"#设置主动吃单
elif (account_id=='19511189162'):
    choosename="中小板（同花顺）"
    tradeway="taker"#设置主动吃单
elif (account_id=='90536336'):
    choosename="大市值（同花顺）"
    tradeway="taker"#设置主动吃单
elif (account_id=='90536337'):
    choosename="微盘股（同花顺）"
    
    # thisnow=datetime.datetime.now().strftime('%m-%d')
    # if (('01-01'<=thisnow)and(thisnow<='01-30')):#一月空仓
    # # if (('04-01'<=thisnow)and(thisnow<='04-30'))or(('01-01'<=thisnow)and(thisnow<='01-30')):#一月四月空仓
    #     # choosename=""#空仓
    #     choosename="大市值（同花顺）"
    # else:
    #     choosename="微盘股（同花顺）"
    
    tradeway="taker"#设置主动吃单
    # tradeway="maker"#设置被动吃单
print(now,choosename,account_id,start_date,last_date,today,yesterday)
choose_stocks(choosename,now,start_date,last_date,today,yesterday)#使用特定函数根据策略名称配置相应参数

buyfilename=choosename+"买入.csv"
sellfilename=choosename+"卖出.csv"
print(buyfilename,sellfilename)
buydf=pd.read_csv(str(start_date)+buyfilename)
selldf=pd.read_csv(str(start_date)+sellfilename)
#确认买入数量【即持仓数量】
targetnum=len(buydf)
print("预计持仓只数",targetnum)

trade_api=TradeAPI(account_id=account_id)#账户登录
portfolio=trade_api.portfolio#获取资产信息并配置参数
available_cash=portfolio["available_cash"]#available_cash	可用资金
market_value=portfolio["market_value"]#market_value	证券市值
frozen_cash=portfolio["frozen_cash"]#frozen_cash	冻结资金
total_value=portfolio["total_value"]#total_value	总资产
print("******","可用资金"+str(available_cash)+"证券市值"+str(market_value)+"冻结资金"+str(frozen_cash)+"总资产"+str(total_value))
premoney=(total_value)/targetnum#确定每只股票的交易金额（根据目标持仓数量制定）

#同花顺内打出来的数据（字符串数据）
buydf["代码"]=buydf["代码"].astype(str).str.zfill(6).apply(lambda x:symbol_convert(x)).astype(str)#需要指定类型为字符串
selldf["代码"]=selldf["代码"].astype(str).str.zfill(6).apply(lambda x:symbol_convert(x)).astype(str)#需要指定类型为字符串
print("buydf",buydf)

print("针对涨停标的不进行卖出处理之前selldf",len(selldf))
positions=trade_api.positions#策略持仓确认（目的是初始化调整下单金额）
for symbol,thisposition in positions.items():
    # print("thisposition",thisposition)
    if thisposition["amount"]>0:
        if (thisposition["symbol"].startswith("00"))or(thisposition["symbol"].startswith("60"))or(thisposition["symbol"].startswith("30")):#只处理主板创业板的股票
            symbol=thisposition["symbol"]
            thisnow=datetime.datetime.now()-datetime.timedelta(seconds=timeseconds)
            # print("symbol",symbol,type(symbol),thisnow)
            try:
                tick=get_tick(securities=symbol,start_date=thisnow,end_date="now",fields=["a1_p","a1_v","a2_p","a2_v","b1_p","b1_v","b2_p","b2_v","current","trade_date"])
                tick=tick.iloc[-1]
                ask_price_1=tick.a1_p
                ask_volume_1=tick.a1_v
                ask_price_2=tick.a2_p
                ask_volume_2=tick.a2_v
                if (ask_price_2==0)and(ask_price_1==0):
                    print(symbol,"涨停不进行卖出")
                    if symbol not in selldf["代码"].tolist():
                        selldf=pd.concat([selldf,pd.DataFrame({"代码":[symbol],"排名":[0]})])
            except Exception as e:#报索引越界一般是tick数据没出来
                print("******","发生bug:",symbol,e)
print("针对涨停标的不进行卖出处理之后selldf",len(selldf))

print("针对跌停标的不进行买入处理之前buydf",len(buydf))
for symbol in buydf["代码"].tolist():
    thisnow=datetime.datetime.now()-datetime.timedelta(seconds=timeseconds)
    # print("symbol",symbol,type(symbol),thisnow)
    try:
        tick=get_tick(securities=symbol,start_date=thisnow,end_date="now",fields=["a1_p","a1_v","a2_p","a2_v","b1_p","b1_v","b2_p","b2_v","current","trade_date"])
        tick=tick.iloc[-1]
        ask_price_1=tick.a1_p
        ask_volume_1=tick.a1_v
        ask_price_2=tick.a2_p
        ask_volume_2=tick.a2_v
        if (ask_price_2==0)and(ask_price_1==0):
            print(symbol,"涨停不进行买入")
            buydf=buydf[~(buydf["代码"]==symbol)]
        bid_price_1=tick.b1_p
        bid_volume_1=tick.b1_v
        bid_price_2=tick.b2_p
        bid_volume_2=tick.b2_v
        if (bid_price_2==0)and(bid_price_1==0):
            print(symbol,"跌停不进行买入")
            buydf=buydf[~(buydf["代码"]==symbol)]
    except Exception as e:#报索引越界一般是tick数据没出来
        print("******","发生bug:",symbol,e)
print("针对跌停标的不进行买入处理之后buydf",len(buydf))

targetcolumn="排名"
dfone=buydf.copy()
dftwo=selldf.copy()
buydf=buydf[["代码",targetcolumn]]
selldf=selldf[["代码",targetcolumn]]
buydf["moneymanage"]=premoney
moneymanage=buydf[["代码","moneymanage"]]
ordernum=0#初始化当前交易轮次为0
print("策略启动","account_id",account_id,"premoney",premoney)

#重置并获取持仓信息【这里的目的是重新获取最新持仓以避免执行卖出成功后数据没有更新导致的持仓数量不对的情况】
dfposition=pd.DataFrame([])
positions=trade_api.positions
vb_list=[
    "symbol",
    "name",#名称
    "amount",
    "frozen_amount",#冻结数量
    "available_amount",#可用数量
    "market_value",#持仓市值
    "cost_basis",#成本价
    "last_price",#当前价格
    'pnl',#盈亏金额
    "profit_rate",#收益率
]
for symbol,info in positions.items():
    if info["amount"]!=0:
        temp=pd.DataFrame([[info[vb] for vb in vb_list]],columns=vb_list,index=[0])
        dfposition=pd.concat([dfposition,temp],ignore_index=True)
dfposition.to_csv("_dfposition.csv")
#判断交易计划
selldflist=dftwo["代码"].tolist()
buydflist=dfone["代码"].tolist()

# 获取当前时间
thistime=datetime.datetime.now()

# #【设置1、4月空仓】【一般不执行或者通过调仓执行】
# if ((thistime.month==4)or(thistime.month==1)):
#     log.info(thistime.month,"当前月份空仓")
#     selldflist=[]
#     buydflist=[]

if not dfposition.empty:#持仓不为空值
    positionsymbols=dfposition["symbol"].tolist()
    falsesymbol=[x for x in positionsymbols if x not in selldflist]
    truesymbol=[x for x in positionsymbols if x in selldflist]
    havesymbol=[x for x in buydflist if x in positionsymbols]
    nothavesymbol=[x for x in buydflist if x not in positionsymbols]
    print("******",
        "不在卖出观察池的需卖出标的",
        falsesymbol,
        "在卖出观察池的正确持仓标的",
        truesymbol,
        "在买入观察池的已持仓标的",
        havesymbol,
        "在买入观察池的未持仓标的",
        nothavesymbol,
        "持仓标的",
        positionsymbols,
    )
    selldf=dfposition.copy()#只针对持仓当中的标的筛选应卖出标的
    buydf=dfone.copy()
    selldf=selldf[~(selldf["symbol"].isin(dftwo["代码"].tolist()))]
    print("实际应卖出股票，去掉应持有标的后",len(selldf))
    # selldf=selldf[~(selldf["symbol"].isin(upstocks))]
    print("实际应卖出股票，去掉涨停标的后",len(selldf))
    if len(selldf)>0:
        #应买入股票处理
        buydf=buydf[~(buydf["代码"].isin(selldf["symbol"].tolist()))]
        buydf=buydf[~(buydf["代码"].isin(dfposition["symbol"].tolist()))]
        print("实际应买入股票，去除应卖出标的后",len(buydf))
        #计算卖出后剩余持仓数量
        hodlstocks=len(dfposition["symbol"].tolist())-len(selldf["symbol"].tolist())
        print("卖出后剩余持仓数量",hodlstocks)
        if hodlstocks!=0:
            buydf=buydf.sort_values(by=targetcolumn)
            buydf=buydf[:(targetnum-hodlstocks)]#这里减去的是持仓股票数量，然后在持仓标的中选择金额不足的向上拼接
            print("对买入计划重新配置之后",len(buydf))
    else:
        print("应卖出股票只数小于0，直接去除掉当前的持仓标的计算买入计划")
        #应买入股票处理
        buydf=buydf[~(buydf["代码"].isin(dfposition["symbol"].tolist()))]
        print("实际应买入股票，去除应卖出标的后",len(buydf))
        #计算卖出后剩余持仓数量
        hodlstocks=len(dfposition["symbol"].tolist())-len(selldf["symbol"].tolist())
        print("全部卖出后剩余持仓数量",hodlstocks)
        if hodlstocks!=0:
            buydf=buydf.sort_values(by=targetcolumn)
            buydf=buydf[:(targetnum-hodlstocks)]#这里减去的是持仓股票数量，然后在持仓标的中选择金额不足的向上拼接
            print("对买入计划重新配置之后",len(buydf))
else:
    if (len(selldflist)==0)and(len(buydflist)==0):
        print(thistime.month,"当月空仓")
        selldf=pd.DataFrame({"代码":[],"总市值":[]})
        buydf=pd.DataFrame({"代码":[],"总市值":[]})
    else:
        print(thistime.month,"正常交易")
        selldf=dftwo.copy()
        buydf=dfone.copy()
    print("实际应卖出股票",len(selldf))
    print("实际应买入股票",len(buydf))
selldf=selldf.reset_index(drop=True)
buydf=buydf.reset_index(drop=True)
print("实际卖出计划",selldf,"实际买入计划",buydf)

#进行交易计划之前的资金管理机制【计算需要对哪些进行买入对哪些进行卖出】
premoney=(total_value)/targetnum#每股理论应持仓金额
#注意这个金额还得补之前超跌的股票的部分的差额
buydf["moneymanage"]=premoney
moneymanage=buydf[["代码","moneymanage"]]
print("单股金额",premoney,"moneymanage",moneymanage)
if not dfposition.empty:
    holddf=dfposition.copy()
    holddf=holddf[~(holddf["symbol"].isin(selldf["symbol"].tolist()))]
    for index,thisposition in holddf.iterrows():#余额不为零才进行下一步免得浪费时间
        # print(index,thisposition)
        symbol=thisposition["symbol"]
        # print("symbol",symbol)
        if thisposition["amount"]>0:#只对当前持仓大于0的标的进行处理
            thispositionmoney=thisposition["market_value"]
            # if (premoney-thispositionmoney)>float(0.0000001)*premoney:#持仓标的与其总资产平均后的理论应持仓市值的偏差在百分之十以上才执行
            if (premoney-thispositionmoney)>float(0.1)*premoney:#持仓标的与其总资产平均后的理论应持仓市值的偏差在百分之十以上才执行
                print(symbol,"premoney",thispositionmoney,"premoney",premoney,"持仓标的与其总资产平均后的理论应持仓市值的偏差在百分之十以上执行补仓操作")
                if symbol not in moneymanage["代码"].tolist():
                    newdata=pd.DataFrame([{"代码":symbol,"moneymanage":(premoney-thispositionmoney)}])
                    moneymanage=pd.concat([moneymanage,newdata],ignore_index=True)
                    print("******","拼接上之前应买入未买全的股票，之后最新的下单金额计划",moneymanage)
                elif symbol in moneymanage["代码"].tolist():
                    moneymanage.loc[moneymanage["代码"]==str(symbol),"moneymanage"]=(premoney-thispositionmoney)
                    print("******","更新完之前应买入未买全的股票，之后最新的下单金额计划",moneymanage)
    moneymanage=moneymanage[moneymanage["moneymanage"]>=targetmoney]#只保留应下单金额大于targetmoney的标的
else:
    print("当前没有持仓，无需对下单计划进行调整")
print(moneymanage)

#初始化存储已经撤销订单的列表【只初始化一次，不要重置】
dfordercancelled=pd.DataFrame({})
while True:
    dforderalls=pd.DataFrame({})#初始化存储全部订单的列表【每一轮都可以重置】
    ordernum+=1#交易轮次计数，避免频繁撤单
    if ordernum>1:
        print(datetime.datetime.now(),"从第二轮开始每执行一轮休息5秒避免订单过度冲击市场","当前轮次",ordernum)
        time.sleep(5)#休息一秒，避免空转
    if ordernum%10==0:
        print("交易轮次达标，执行撤单任务")
        if cancellorder:#如果cancellorder设置为true则执行以下撤单流程【最低撤单金额一万元】
            orderalls=trade_api.get_orders()#获取全部订单，针对已经撤销的订单进行处理【理论上金额处理部分应该将下单计划暂停，但是由于这里是在同一个线程执行所以不用单独设置】
            for orderall in orderalls:
                if ((orderall.status!="全部成交")and(orderall.status!="已初始化")):#拼接orderall的数据【不对全部成交或者已初始化的订单进行处理】
                    dforderall=pd.DataFrame({
                            "order_id":[orderall.order_id],
                            "status":[orderall.status],
                            "symbol":[orderall.symbol],
                            "amount":[orderall.amount],
                            "cancel_amount":[orderall.cancel_amount],
                            "trade_amount":[orderall.trade_amount],
                            "trade_price":[orderall.trade_price],
                            "side":[orderall.side],
                            "price":[orderall.price],
                            "price_type":[orderall.price_type],
                            "datetime":[orderall.datetime],
                            "secondary_order_id":[orderall.secondary_order_id]})
                    dforderalls=pd.concat([dforderalls,dforderall],ignore_index=True)
                    if ((orderall.status=="部分成交")or(orderall.status=="未成交")):
                        print("******","不是已成交订单",orderall.order_id)
                        #60秒内不成交就撤单【这个是要小于当前时间，否则就一直无法执行】
                        if (orderall.datetime+datetime.timedelta(seconds=timecancellwait))<datetime.datetime.now():#成交额还得超过targetmoney才可以最终撤单
                            if (orderall.trade_amount*orderall.price>targetmoney):
                                try:
                                    trade_api.cancel_order(orderall.order_id)
                                    print("******","已成交金额达标执行撤单",orderall.order_id)
                                except:
                                    print("******","已完成或取消中的条件单不允许取消")
                            elif orderall.trade_amount==0:#未成交撤单
                                try:#如果该委托已成交或者已撤单则会报错
                                    trade_api.cancel_order(orderall.order_id)
                                    print("******",'执行撤单',orderall.order_id)
                                except:
                                    print("******",'已完成或取消中的条件单不允许取消')
                    else:#撤单或者废单之后的金额回补
                        if (orderall.side=="BUY"):#这里只计算BUY方向的订单
                            if (orderall.status=="已撤单"):
                                print("******","撤单成功",orderall,orderall.status,orderall.cancel_amount)
                                if dfordercancelled.empty:#dfordercancelled一开始是个空值，这里主要是确认一下之前有没有数据，有数据才需要检验之前是否撤销过
                                    dfordercancelled=pd.concat([dfordercancelled,dforderall],ignore_index=True)
                                    cancel_money=orderall.cancel_amount*orderall.price#然后就是计算撤销了的订单的未完成金额，加给下单金额当中
                                    moneymanage.loc[moneymanage["代码"]==str(orderall.symbol),"moneymanage"]+=cancel_money
                                else:
                                    if orderall.order_id not in dfordercancelled["order_id"].tolist():
                                        dfordercancelled=pd.concat([dfordercancelled,dforderall],ignore_index=True)
                                        cancel_money=orderall.cancel_amount*orderall.price#然后就是计算撤销了的订单的未完成金额，加给下单金额当中
                                        moneymanage.loc[moneymanage["代码"]==str(orderall.symbol),"moneymanage"]+=cancel_money
                            elif (orderall.status=="废单"):
                                print("******","废单处理",orderall,orderall.status,orderall.amount)
                                if dfordercancelled.empty:#dfordercancelled一开始是个空值，这里主要是确认一下之前有没有数据，有数据才需要检验之前是否撤销过
                                    dfordercancelled=pd.concat([dfordercancelled,dforderall],ignore_index=True)
                                    cancel_money=orderall.amount*orderall.price#然后就是计算撤销了的订单的未完成金额，加给下单金额当中
                                    moneymanage.loc[moneymanage["代码"]==str(orderall.symbol),"moneymanage"]+=cancel_money
                                if orderall.order_id not in dfordercancelled["order_id"].tolist():
                                    dfordercancelled=pd.concat([dfordercancelled,dforderall],ignore_index=True)
                                    cancel_money=orderall.amount*orderall.price#然后就是计算撤销了的订单的未完成金额，加给下单金额当中
                                    moneymanage.loc[moneymanage["代码"]==str(orderall.symbol),"moneymanage"]+=cancel_money
            dforderalls.to_csv("_dforderalls.csv")#输出所有未全部成交的订单【针对所有订单】
            dfordercancelled.to_csv("_dfordercancelled.csv")#输出已经撤销或者作废的订单【只针对的买入订单】
            print("******","资金管理","premoney",premoney,"moneymanage",moneymanage)
    moneymanage.to_csv("_dfmoneymanage.csv")
    #重置并获取持仓信息【这里的目的是重新获取最新持仓以避免执行卖出成功后数据没有更新导致的持仓数量不对的情况】
    dfposition=pd.DataFrame([])
    positions=trade_api.positions
    vb_list=[
        "symbol",
        "name",#名称
        "amount",
        "frozen_amount",#冻结数量
        "available_amount",#可用数量
        "market_value",#持仓市值
        "cost_basis",#成本价
        "last_price",#当前价格
        'pnl',#盈亏金额
        "profit_rate",#收益率
    ]
    for symbol,info in positions.items():
        if info["amount"]!=0:
            temp=pd.DataFrame([[info[vb] for vb in vb_list]],columns=vb_list,index=[0])
            dfposition=pd.concat([dfposition,temp],ignore_index=True)
    dfposition.to_csv("_dfposition.csv")
    print("******","卖出")
    for symbol in dfposition["symbol"].tolist():
        if symbol in selldf["symbol"].tolist():
            thisposition=trade_api.positions[symbol]
            if (thisposition.available_amount>0):#余额及可用余额都要大于0才执行卖出动作
                print("******",symbol,"持仓数量",thisposition.amount,"持仓市值",thisposition.market_value,"可用数量",thisposition.available_amount,"当前价格",thisposition.last_price)
                #历史tick数据（包含时间戳）
                start_date=datetime.datetime.now()-datetime.timedelta(seconds=timeseconds)
                end_date="now"
                try:
                    tick=get_tick(securities=symbol,start_date=start_date,end_date="now",fields=["b1_p","b1_v","a1_p","a1_v","current","trade_date"])
                    tick=tick.iloc[-1]
                    ask_price_1=tick.a1_p
                    ask_volume_1=tick.a1_v
                    bid_price_1=tick.b1_p
                    bid_volume_1=tick.b1_v
                    currentprice=tick.current
                    print(currentprice)
                    if (tick.trade_date+datetime.timedelta(seconds=timetickwait)>datetime.datetime.now()):
                        print("******","确认是最新tick，执行交易")
                        if (ask_price_1-bid_price_1)<=(((ask_price_1+bid_price_1)/2)*bidrate):#盘口价差
                            print("******","盘口价差适宜，适合执行交易")
                            if ((symbol.startswith("12")) or (symbol.startswith("11"))):#针对11开头或者12开头的转债单独处理
                                if tradeway=="maker":#maker下单【不需要考虑深度问题】
                                    if (thisposition.available_amount*ask_price_1)<(traderate*targetmoney):
                                        print("******","剩余全部卖出")
                                        sellvolume=(math.floor(thisposition.available_amount/10)*10)
                                        sellorder=trade_api.order(symbol=symbol,amount=-sellvolume,price=ask_price_1)
                                        log.info(sellorder)
                                    else:#限价卖出最小下单金额
                                        print("******","卖出目标金额")
                                        sellvolume=(math.floor((targetmoney/ask_price_1)/10)*10)
                                        if (thisposition.available_amount*bid_price_1)>500000:#针对余额高于500000的标的单独扩大下单数量
                                            sellvolume*=10
                                        print(sellvolume,sellvolume*ask_price_1)
                                        sellorder=trade_api.order(symbol=symbol,amount=-sellvolume,price=ask_price_1)
                                        log.info(sellorder)
                                    time.sleep(1)
                                if tradeway=="taker":#maker下单【需要考虑深度问题】
                                    if (bid_price_1*bid_volume_1)>targetmoney:#盘口深度【己方一档买入】（转债价格较高，一档深度相对小一些）                                 
                                        if (thisposition.available_amount*bid_price_1)<(traderate*targetmoney):
                                            print("******","剩余全部卖出")
                                            sellvolume=(math.floor(thisposition.available_amount/10)*10)
                                            sellorder=trade_api.order(symbol=symbol,amount=-sellvolume,price=bid_price_1)
                                            log.info(sellorder)
                                        else:#限价卖出最小下单金额
                                            print("******","卖出目标金额")
                                            sellvolume=(math.floor((targetmoney/bid_price_1)/10)*10)
                                            if (thisposition.available_amount*bid_price_1)>500000:#针对余额高于500000的标的单独扩大下单数量
                                                sellvolume*=10
                                            print(sellvolume,sellvolume*bid_price_1)
                                            sellorder=trade_api.order(symbol=symbol,amount=-sellvolume,price=bid_price_1)
                                            log.info(sellorder)
                            else:#非可转债交易方式
                                if tradeway=="maker":#maker下单【不需要考虑深度问题】
                                    if (thisposition.available_amount*ask_price_1)<(traderate*targetmoney):
                                        print("******","剩余全部卖出")
                                        sellvolume=(math.floor(thisposition.available_amount/100)*100)
                                        sellorder=trade_api.order(symbol=symbol,amount=-sellvolume,price=ask_price_1)
                                        log.info(sellorder)
                                    else:#限价卖出最小下单金额
                                        print("******","卖出目标金额")
                                        sellvolume=(math.floor((targetmoney/ask_price_1)/100)*100)
                                        if (thisposition.available_amount*ask_price_1)>500000:#针对余额高于500000的标的单独扩大下单数量
                                            sellvolume*=10
                                        print(sellvolume,sellvolume*ask_price_1)
                                        sellorder=trade_api.order(symbol=symbol,amount=-sellvolume,price=ask_price_1)
                                        log.info(sellorder)
                                if tradeway=="taker":#maker下单【需要考虑深度问题】
                                    if (bid_price_1*bid_volume_1)>targetmoney:#盘口深度【对手盘一档买入】                                            
                                        if (thisposition.available_amount*bid_price_1)<(traderate*targetmoney):
                                            print("******","剩余全部卖出")
                                            sellvolume=(math.floor(thisposition.available_amount/100)*100)
                                            sellorder=trade_api.order(symbol=symbol,amount=-sellvolume,price=bid_price_1)
                                            log.info(sellorder)
                                        else:#限价卖出最小下单金额
                                            print("******","卖出目标金额")
                                            sellvolume=(math.floor((targetmoney/bid_price_1)/100)*100)
                                            if (thisposition.available_amount*bid_price_1)>500000:#针对余额高于500000的标的单独扩大下单数量
                                                sellvolume*=10
                                            print(sellvolume,sellvolume*bid_price_1)
                                            sellorder=trade_api.order(symbol=symbol,amount=-sellvolume,price=bid_price_1)
                                            log.info(sellorder)
                                if tradeway=="集合竞价":                                  
                                    ###当前时间大于9：30执行下面的代码
                                    thisnow=(datetime.datetime.utcnow()+datetime.timedelta(hours=8)).time()#获取标准时间，换算成东八区
                                    print("当前小时分钟数",thisnow,"如果卡在这里了说明未到开盘时间或者同花顺可能修改时区了")
                                    if (thisnow<datetime.time(9,24)):#【如果卡在这里了说明同花顺可能修改时区了】
                                        print("尚未到达9点24等待数据",tick)
                                        time.sleep(1)
                                    elif (tradewaynum==0)and(thisnow>datetime.time(9,24))and(thisnow<datetime.time(9,25)):#【如果卡在这里了说明同花顺可能修改时区了】
                                        print("集合竞价打印tick，此时买一价和卖一价相同、买一量和卖一量相同",tick)
                                        if (bid_price_1*bid_volume_1)>(4*targetmoney):#盘口深度【对手盘一档买入】【后期改成五倍深度才卖出】
                                            print("盘口挂单满足下单需求4*targetmoney")
                                            selltargetprice=(math.ceil(currentprice*(1-0.02)*100)/100)#卖出的时候用这个（向上取整）
                                            if (thisposition.available_amount)<(bid_volume_1/4):
                                                print("集合竞价盘口够深全部金额直接下出")
                                                sellvolume=(math.floor(thisposition.available_amount/100)*100)
                                                print("sellvolume",sellvolume)
                                                sellorder=trade_api.order(symbol=symbol,amount=-sellvolume,price=selltargetprice)
                                                log.info(sellorder)
                                            else:#限价卖出最小下单金额
                                                print("集合竞价盘口够深全部金额直接下出")
                                                sellvolume=(math.floor(bid_volume_1/4/100)*100)
                                                print("sellvolume",sellvolume)
                                                sellorder=trade_api.order(symbol=symbol,amount=-sellvolume,price=selltargetprice)
                                                log.info(sellorder)
                                            tradewaynum+=1
                                    else:
                                        if (thisnow<=datetime.time(9,30)):#9：30之后转taker 
                                            print("尚未到达9点30无法下单",thisnow)
                                            time.sleep(1)
                                        elif (thisnow>datetime.time(9,30)):#9：30之后转taker 
                                            tradeway="taker"#卖出执行成功之后自动切换到taker方式下单
                                            print("正式开盘转taker下单",thisnow)
                except Exception as e:#报索引越界一般是tick数据没出来
                    print("******","发生bug:",symbol,e)
    print("******","买入")
    moneymanage=moneymanage.sort_values(by="moneymanage",ascending=False)#这里是由大到小排序，默认由小到大
    for symbol in moneymanage["代码"].tolist():#如果恰好是三十只以上股票，且没有需要卖出的股票时，moneymanage为空会导致报错
        buymoney=moneymanage[moneymanage["代码"]==str(symbol)]["moneymanage"].iloc[0]
        if buymoney>targetmoney:#只针对待买入金额超过targetmoney的标的进行买入，否则直接掠过
            #重置并获取资产信息
            portfolio_available_cash=trade_api.portfolio["available_cash"]#available_cash	可用资金
            print("当前余额",portfolio_available_cash)
            if portfolio_available_cash>targetmoney:
                print("******","买入余额充足",symbol)
                #历史tick数据（包含时间戳）
                start_date=datetime.datetime.now()-datetime.timedelta(seconds=timeseconds)
                end_date="now"
                try:
                    tick=get_tick(securities=symbol,start_date=start_date,end_date="now",fields=["b1_p","b1_v","a1_p","a1_v","current","trade_date"])
                    tick=tick.iloc[-1]
                    ask_price_1=tick.a1_p
                    ask_volume_1=tick.a1_v
                    bid_price_1=tick.b1_p
                    bid_volume_1=tick.b1_v
                    currentprice=tick.current
                    print(currentprice)
                    if (tradeway=="打板"):
                        thisnow=(datetime.datetime.utcnow()+datetime.timedelta(hours=8)).time()#获取标准时间，换算成东八区
                        print("当前小时分钟数",thisnow,"如果卡在这里了说明未到开盘时间或者同花顺可能修改时区了")
                        if (thisnow>datetime.time(9,34))and(thisnow<datetime.time(15,50)):#【如果卡在这里了说明同花顺可能修改时区了】
                            print("******","买入")
                            #重置并获取资产信息
                            portfolio_available_cash=trade_api.portfolio["available_cash"]#available_cash	可用资金
                            print("当前余额",portfolio_available_cash)
                            if portfolio_available_cash>targetmoney:
                                print("******","买入余额充足",symbol)
                                #历史tick数据（包含时间戳）
                                start_date=datetime.datetime.now()-datetime.timedelta(seconds=timeseconds)
                                end_date="now"
                                tick=get_tick(securities=symbol,start_date=start_date,end_date="now",fields=["b1_p","b1_v","a1_p","a1_v","b2_p","b2_v","a2_p","a2_v","current","trade_date"])
                                tick=tick.iloc[-1]
                                ask_price_2=tick.a2_p
                                ask_volume_2=tick.a2_v
                                bid_price_2=tick.b2_p
                                bid_volume_2=tick.b2_v
                                print("******速度测试",tick.trade_date,type(tick.trade_date),datetime.datetime.now())
                                if (tick.trade_date+datetime.timedelta(seconds=timetickwait)>datetime.datetime.now()):
                                    print("******","确认是最新tick，执行买入")
                                    if (ask_price_2==0)and(ask_price_1!=0):#即将涨停
                                        print("******","即将涨停")
                                        if buymoney<(traderate*targetmoney):
                                            print("******","剩余全部买入")
                                            buyvolume=(math.floor((buymoney/ask_price_1)/10)*10)
                                            buyorder=trade_api.order(symbol=symbol,amount=buyvolume,price=ask_price_1)
                                            log.info(buyorder)
                                            bidmoney=float(ask_price_1)*buyvolume
                                            moneymanage.loc[moneymanage["代码"]==str(symbol),"moneymanage"]-=bidmoney
                                        else:
                                            print("******","买入目标金额")
                                            buyvolume=(math.floor((targetmoney/ask_price_1)/10)*10)
                                            if buymoney>500000:#针对总下单余额高于500000的标的单独扩大下单数量
                                                buyvolume*=10
                                            buyorder=trade_api.order(symbol=symbol,amount=buyvolume,price=ask_price_1)
                                            log.info(buyorder)
                                            bidmoney=float(ask_price_1)*buyvolume
                                            moneymanage.loc[moneymanage["代码"]==str(symbol),"moneymanage"]-=bidmoney
                    elif (tradeway!="打板"):
                        print("******速度测试",tick.trade_date,type(tick.trade_date),datetime.datetime.now())
                        if (tick.trade_date+datetime.timedelta(seconds=timetickwait)>datetime.datetime.now()):
                            print("******","确认是最新tick，执行买入")
                            if (ask_price_1-bid_price_1)<=(((ask_price_1+bid_price_1)/2)*bidrate):#盘口价差
                                print("******","盘口价差适宜，适合执行交易")        
                                if ((symbol.startswith("12")) or (symbol.startswith("11"))):#针对11开头或者12开头的转债单独处理
                                    if tradeway=="maker":#maker下单【不需要考虑深度问题】
                                        if buymoney<(traderate*targetmoney):
                                            print("******","剩余全部买入")
                                            buyvolume=(math.floor((buymoney/bid_price_1)/10)*10)
                                            buyorder=trade_api.order(symbol=symbol,amount=buyvolume,price=bid_price_1)
                                            log.info(buyorder)
                                            bidmoney=float(bid_price_1)*buyvolume
                                            moneymanage.loc[moneymanage["代码"]==str(symbol),"moneymanage"]-=bidmoney
                                        else:
                                            print("******","买入目标金额")
                                            buyvolume=(math.floor((targetmoney/bid_price_1)/10)*10)
                                            if buymoney>500000:#针对总下单余额高于500000的标的单独扩大下单数量
                                                buyvolume*=10
                                            buyorder=trade_api.order(symbol=symbol,amount=buyvolume,price=bid_price_1)
                                            log.info(buyorder)
                                            bidmoney=float(bid_price_1)*buyvolume
                                            moneymanage.loc[moneymanage["代码"]==str(symbol),"moneymanage"]-=bidmoney
                                        time.sleep(1)
                                    if tradeway=="taker":#taker下单【跟其他地方一样需要考虑深度】
                                        if (ask_price_1*ask_volume_1)>targetmoney:#盘口深度【己方一档买入】（转债价格较高，一档深度相对小一些） 
                                            if buymoney<(traderate*targetmoney):
                                                print("******","剩余全部买入")
                                                buyvolume=(math.floor((buymoney/ask_price_1)/10)*10)
                                                buyorder=trade_api.order(symbol=symbol,amount=buyvolume,price=ask_price_1)
                                                log.info(buyorder)
                                                bidmoney=float(ask_price_1)*buyvolume
                                                moneymanage.loc[moneymanage["代码"]==str(symbol),"moneymanage"]-=bidmoney
                                            else:
                                                print("******","买入目标金额")
                                                buyvolume=(math.floor((targetmoney/ask_price_1)/10)*10)
                                                if buymoney>500000:#针对总下单余额高于500000的标的单独扩大下单数量
                                                    buyvolume*=10
                                                buyorder=trade_api.order(symbol=symbol,amount=buyvolume,price=ask_price_1)
                                                log.info(buyorder)
                                                bidmoney=float(ask_price_1)*buyvolume
                                                moneymanage.loc[moneymanage["代码"]==str(symbol),"moneymanage"]-=bidmoney
                                else:#其他交易情况
                                    if tradeway=="maker":#maker下单【不需要考虑深度问题】
                                        if buymoney<(traderate*targetmoney):
                                            print("******","剩余全部买入")
                                            buyvolume=(math.floor((buymoney/bid_price_1)/100)*100)
                                            buyorder=trade_api.order(symbol=symbol,amount=buyvolume,price=bid_price_1)
                                            log.info(buyorder)
                                            bidmoney=float(bid_price_1)*buyvolume
                                            moneymanage.loc[moneymanage["代码"]==str(symbol),"moneymanage"]-=bidmoney
                                        else:
                                            print("******","买入目标金额")
                                            buyvolume=(math.floor((targetmoney/bid_price_1)/100)*100)
                                            if buymoney>500000:#针对总下单余额高于500000的标的单独扩大下单数量
                                                buyvolume*=10
                                            buyorder=trade_api.order(symbol=symbol,amount=buyvolume,price=bid_price_1)
                                            log.info(buyorder)
                                            bidmoney=float(bid_price_1)*buyvolume
                                            moneymanage.loc[moneymanage["代码"]==str(symbol),"moneymanage"]-=bidmoney
                                    if tradeway=="taker":#taker下单【跟其他地方一样需要考虑深度】
                                        if (ask_price_1*ask_volume_1)>targetmoney:#盘口深度【对手盘一档买入】
                                            if buymoney<(traderate*targetmoney):
                                                print("******","剩余全部买入")
                                                buyvolume=(math.floor((buymoney/ask_price_1)/100)*100)
                                                buyorder=trade_api.order(symbol=symbol,amount=buyvolume,price=ask_price_1)
                                                log.info(buyorder)
                                                bidmoney=float(ask_price_1)*buyvolume
                                                moneymanage.loc[moneymanage["代码"]==str(symbol),"moneymanage"]-=bidmoney
                                            else:
                                                print("******","买入目标金额")
                                                buyvolume=(math.floor((targetmoney/ask_price_1)/100)*100)
                                                if buymoney>500000:#针对总下单余额高于500000的标的单独扩大下单数量
                                                    buyvolume*=10
                                                buyorder=trade_api.order(symbol=symbol,amount=buyvolume,price=ask_price_1)
                                                log.info(buyorder)
                                                bidmoney=float(ask_price_1)*buyvolume
                                                moneymanage.loc[moneymanage["代码"]==str(symbol),"moneymanage"]-=bidmoney
                                    if tradeway=="集合竞价":
                                        print("卖出时直接taker交易")
                                        tradeway="taker"
                except Exception as e:#报索引越界一般是tick数据没出来
                    print("******","发生bug:",symbol,e)
    print("******","任务结束")
    
# # 函数声明：【同步同花顺APP自选板块设置】
# # custom_sector(name,action="query",symbol=None,rename=None)
# name="supermind板块"
# message=custom_sector(name,action="remove")#删除name板块
# message=custom_sector(name,action="insert",symbol=["000001.SZ","000002.SZ"])#新建name板块
# # message=custom_sector(name,action="update",symbol=["000001.SZ"],rename="supermind")#更新name板块的股票列表或名字
# # message=custom_sector(name,action="append",symbol=["000001.SZ","000002.SZ"])
# # message=custom_sector(name,action="pop",symbol=["000001.SZ"])#移除板块1的股票
# # message=custom_sector()#返回所有板块的dataframe
# # message=custom_sector(name)#返回所有板块的dataframe
# print(message)


2024-05-13 2023-09-06
****** 等待策略初始化 20240513 20240510
2024-05-13 09:39:13.224234 中小板（同花顺） 19511189162 2024-05-13 2023-09-06 20240513 20240510
****** ******2024-05-13中小板（同花顺）买入.csv中小板（同花顺）卖出.csv文件不同时存在
修改前 5363
修改后 5187
4370
****** ['002883.SZ' '002830.SZ' '003017.SZ' '002295.SZ' '002836.SZ' '003011.SZ'] ['002883.SZ' '002830.SZ' '003017.SZ' '002295.SZ' '002836.SZ' '003011.SZ'
 '002718.SZ' '002890.SZ' '003003.SZ']
中小板（同花顺）买入.csv 中小板（同花顺）卖出.csv
预计持仓只数 6
****** 可用资金1638.34证券市值211149.0冻结资金0.0总资产212787.34
buydf    Unnamed: 0         代码          总股本        归母净利润                major  \
0           0  002883.SZ  156157166.0   9959464.30  2024-05-13 09:37:00   
1           1  002830.SZ  133340000.0   3684345.55  2024-05-13 09:37:00   
2           2  003017.SZ   84000000.0  16918222.12  2024-05-13 09:37:00   
3           5  002295.SZ  250616000.0   7239796.61  2024-05-13 09:37:00   
4           3  002836.SZ  230400000.0  10588916.36  2024-05-13 09:37:00   
5           4  003011.SZ  102676000.0 

KeyboardInterrupt: 